# Warhammer RAG

## Installation

In [1]:
!python -m pip install --upgrade pip
!pip install llama-cpp-python
!pip install langchain langchain-community sentence-transformers chromadb
!pip install pypdf requests pydantic tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 79.9 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4067755 sha256=8c1b2e92115f08a270d882f06f13f63a5b8675d2f8e88b59b5c5bfbfbf3e3444
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama-cpp-python
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [llama-cpp-python][llama-cpp-python]
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
 

## Imports

In [41]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import LlamaCpp
from tqdm import tqdm

import os
import re
import time
import requests
import tiktoken

## Data Preprocessing

In [3]:
# Load Data
data_path = '/kaggle/input/warhammer-4e-rpg/WFRP_4_ed_PL_1.3.pdf'

loader = PyPDFLoader(data_path)
wh_rulebook = loader.load()

In [ ]:
('page_content', 'WARHAMMER FANTASY ROLEPLAY\n6\nWPROWADZENIE•\nWitajcie w grze Warhammer Fantasy Roleplay. Czekają na was \nwspaniałe rzeczy. Jedna osoba z waszej grupy będzie kontrolowała \ncały świat, podczas gdy reszta z was będzie w nim żyła i pozna -\nwała go, odkrywając pełne blasku cudowności, plugawą ciemność \ni wszystko pomiędzy nimi.\nSpotkacie znamienitych bohaterów, którzy z wyrazem pogardy omi-\njają cierpiącą biedotę i zdeprawowanych łotrów, którzy służąc chwa-\nlebnej sprawie, zboczyli na złą drogę. Budzący lęk, lecz także respekt, \nczarodzieje doskonalą swoją sztukę w wysokich wieżach Kolegiów \nMagii. Tymczasem ci, którzy praktykują magię poza bezpiecznymi \nmurami Kolegiów, wywołują strach i są piętnowani – często nie bez \npowodu, gdyż wielu mrocznych czarnoksiężników ochoczo sprzeda-\nłoby duszę w zamian za potęgę. Cnotliwi kapłani nieustannie starają \nsię polepszyć los uciśnionych, podczas gdy agenci Bogów Chaosu \npodkopują to dobre dzieło i sprowadzają zniszczenie.\nSzykujcie się na zmagania w rynsztokach, na walkę o przetrwanie \ni na zmierzenie się z zepsuciem, które napiera zewsząd. Przygotuj -\ncie się na zagrożenia, te czyhają bowiem wszędzie, a nie stawicie im \nczoła w pojedynkę. Przede wszystkim jednak nastawcie się na ponure \ni niebezpieczne przygody w grze Warhammer Fantasy Roleplay!\nGry fabularne\nWarhammer Fantasy Roleplay (WFRP) to gra fabularna, określa -\nna też skrótem RPG, z angielskiego „roleplaying game”. Być może \nlepiej znane są wam gry RPG z komputerów lub konsol. W takim \nprzypadku od razu poczujecie się jak u siebie. Jedna osoba z waszej \ngrupy przyjmie rolę Mistrza Gry (MG), który opisuje świat i wszyst-\nko, co się dzieje. Pozostali będą Graczami – protagonistami gry, \nktórzy dokonują interakcji ze światem przedstawianym przez MG. \nGracze mówią MG, co robią ich Bohaterowie, a MG interpretuje \nrezultaty tych działań, w razie potrzeby posiłkując się zasadami gry. \nI\n•\nA zatem, czego tu szukasz? Przygody?\nByć może. Złota? Zapewne. Sprawiedliwości?\nHa, to dość względne pojęcie! Cóż to? Świętoszkowaty błysk w twym oku? A, chcesz robić \nto, co jest właściwe... Dopóki jest to dobrze płatne, dostarcza Ci rozrywki i pasuje do Twoich \npoglądów. Niech będzie, to wystarczy. Nadasz się. Wejdź, opowiem Ci o tej robocie.\nGdy wymagany jest rezultat losowy, gra WFRP korzysta \nz dziesięciościennej kostki. Dziesięciościenne kostki zazwyczaj \nmają ścianki oznakowane od 0 do 9, gdzie rzut 0 liczy się jako \n10. W zasadach takie kości określane są jako k10, a ich liczba, \nktórą trzeba rzucić, zawsze jest podana w następujący sposób: \n1k10 za jedną kostkę, 2k10 za dwie kostki, 3k10 za trzy kostki \ni tak dalej.\nJeśli należy rzucić kilkoma kośćmi, wyniki są zawsze \nsumowane. Zatem jeśli zasady proszą o rzucenie 2k10, rzucasz \ndwoma dziesięciościennymi kośćmi i dodajesz wyniki ich obu, \nna przykład rzut 0 i 3 oznacza wynik 13 (10+3=13).\nCzasami rzut kostką zostanie zmodyfikowany przez dodanie \nlub odjęcie liczby. Zatem rzut 1k10+4 oznacza rzut jedną \ndziesięciościenną kostką i dodanie do wyniku 4, natomiast rzut \n2k10-3 wskazuje, że należy rzucić dwoma dziesięciościennymi \nkośćmi i odjąć od sumarycznego wyniku 3.\nPonadto zasady wykorzystują rzut dwoma dziesięciościennymi \nkośćmi do uzyskania liczby od 1 do 100 (oznakowane jako \n1k100). Aby to zrobić, jedna dziesięciościenna kostka zostaje \nuznana za kość „dziesiątek”, a druga za kość „jedności”. Teraz \nrzuć dwoma kośćmi i odczytaj wynik jako liczbę dwucyfrową. \nUwaga, w tym przypadku wynik „0” na kostce odczytujemy \nzawsze właśnie jako zero! Zatem rzut 1 na kostce dziesiątek i 4 \nna kostce jedności daje wynik 14, a rzut 4 i 2 oznacza 42. Jeśli \nna obu kościach wypadło 0, wynik wynosi 100.\nKOŚCI ZOSTAŁY RZUCONE\nTomasz Otto (Order #44833549)')

In [22]:
wh_rulebook_raw = ''

for page in wh_rulebook:
    wh_rulebook_raw += page.page_content + '/n'

In [73]:
pattern = r"Wprowadzenie"
start = re.finditer(pattern, wh_rulebook_raw)

In [74]:
for i in start:
    print(i)

<re.Match object; span=(1329492, 1329504), match='Wprowadzenie'>


In [51]:
wh_rulebook_raw[start-10:start+1000]

'.....  145\nWprowadzenie ....................................  6\nWrażliwe punkty (pancerz) ..............  300\nWrogość ..................................  191, 342\nWrogość: opcjonalna krasnoludów  \ndo elfów .............................................  26\nWrota ziemi (zaklęcie) .....................  253\nWróżba Losu ...................................  146\nWróżono nam zgubę .......................  146\nWrzeszcząca czaszka (zaklęcie) ....... 257\nWsie, Sioła i Miejsca Święte ............ 285\nWspaniały Reikland ........................  266\nWspinacz (stworzenia) ....................  342\nWspinaczka .............................  130, 166\nWstrętni Szczuroludzie ...................  336\nWstrzemięźliwy ...............................  146\nWstrzymanie dłoni Morra (cud) ..... 223\nWściekły Atak .................................  146\nWtargnięcie z Włamaniem ..............  146\nWyborny Wspinacz .........................  146\nWybór własnego wyposażenia: zasada \nopcjonalna ...

In [ ]:
# Split data into chunks

separators = [
r"\n.*?•\n", # New chapter
r"\n.*?\n",  # New topic
r"\n\n",     # Paragraphs
r"\n",       # Lines
r".",        # Sentences
]

encoding = tiktoken.get_encoding("cl100k_base")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    separators=separators,
    length_function=lambda x: len(encoding.encode(x)),
    is_separator_regex=True,
    keep_separator=True
)

chunks = text_splitter.split_documents(wh_rulebook)

In [ ]:
('page_content', 'WARHAMMER FANTASY ROLEPLAY\n6')

In [112]:
len(chunks)

1276

In [ ]:
('page_content', 'WARHAMMER FANTASY ROLEPLAY\n6\nWPROWADZENIE•\nWitajcie w grze Warhammer Fantasy Roleplay. Czekają na was \nwspaniałe rzeczy. Jedna osoba z waszej grupy będzie kontrolowała \ncały świat, podczas gdy reszta z was będzie w nim żyła i pozna -\nwała go, odkrywając pełne blasku cudowności, plugawą ciemność \ni wszystko pomiędzy nimi.\nSpotkacie znamienitych bohaterów, którzy z wyrazem pogardy omi-\njają cierpiącą biedotę i zdeprawowanych łotrów, którzy służąc chwa-\nlebnej sprawie, zboczyli na złą drogę. Budzący lęk, lecz także respekt, \nczarodzieje doskonalą swoją sztukę w wysokich wieżach Kolegiów \nMagii. Tymczasem ci, którzy praktykują magię poza bezpiecznymi \nmurami Kolegiów, wywołują strach i są piętnowani – często nie bez \npowodu, gdyż wielu mrocznych czarnoksiężników ochoczo sprzeda-\nłoby duszę w zamian za potęgę. Cnotliwi kapłani nieustannie starają \nsię polepszyć los uciśnionych, podczas gdy agenci Bogów Chaosu \npodkopują to dobre dzieło i sprowadzają zniszczenie.\nSzykujcie się na zmagania w rynsztokach, na walkę o przetrwanie \ni na zmierzenie się z zepsuciem, które napiera zewsząd. Przygotuj -\ncie się na zagrożenia, te czyhają bowiem wszędzie, a nie stawicie im \nczoła w pojedynkę. Przede wszystkim jednak nastawcie się na ponure \ni niebezpieczne przygody w grze Warhammer Fantasy Roleplay!\nGry fabularne\nWarhammer Fantasy Roleplay (WFRP) to gra fabularna, określa -\nna też skrótem RPG, z angielskiego „roleplaying game”. Być może \nlepiej znane są wam gry RPG z komputerów lub konsol. W takim \nprzypadku od razu poczujecie się jak u siebie. Jedna osoba z waszej \ngrupy przyjmie rolę Mistrza Gry (MG), który opisuje świat i wszyst-\nko, co się dzieje. Pozostali będą Graczami – protagonistami gry, \nktórzy dokonują interakcji ze światem przedstawianym przez MG. \nGracze mówią MG, co robią ich Bohaterowie, a MG interpretuje \nrezultaty tych działań, w razie potrzeby posiłkując się zasadami gry. \nI\n•\nA zatem, czego tu szukasz? Przygody?\nByć może. Złota? Zapewne. Sprawiedliwości?\nHa, to dość względne pojęcie! Cóż to? Świętoszkowaty błysk w twym oku? A, chcesz robić \nto, co jest właściwe... Dopóki jest to dobrze płatne, dostarcza Ci rozrywki i pasuje do Twoich \npoglądów. Niech będzie, to wystarczy. Nadasz się. Wejdź, opowiem Ci o tej robocie.\nGdy wymagany jest rezultat losowy, gra WFRP korzysta \nz dziesięciościennej kostki. Dziesięciościenne kostki zazwyczaj \nmają ścianki oznakowane od 0 do 9, gdzie rzut 0 liczy się jako \n10. W zasadach takie kości określane są jako k10, a ich liczba, \nktórą trzeba rzucić, zawsze jest podana w następujący sposób: \n1k10 za jedną kostkę, 2k10 za dwie kostki, 3k10 za trzy kostki \ni tak dalej.\nJeśli należy rzucić kilkoma kośćmi, wyniki są zawsze \nsumowane. Zatem jeśli zasady proszą o rzucenie 2k10, rzucasz \ndwoma dziesięciościennymi kośćmi i dodajesz wyniki ich obu, \nna przykład rzut 0 i 3 oznacza wynik 13 (10+3=13).\nCzasami rzut kostką zostanie zmodyfikowany przez dodanie \nlub odjęcie liczby. Zatem rzut 1k10+4 oznacza rzut jedną \ndziesięciościenną kostką i dodanie do wyniku 4, natomiast rzut \n2k10-3 wskazuje, że należy rzucić dwoma dziesięciościennymi \nkośćmi i odjąć od sumarycznego wyniku 3.\nPonadto zasady wykorzystują rzut dwoma dziesięciościennymi \nkośćmi do uzyskania liczby od 1 do 100 (oznakowane jako \n1k100). Aby to zrobić, jedna dziesięciościenna kostka zostaje \nuznana za kość „dziesiątek”, a druga za kość „jedności”. Teraz \nrzuć dwoma kośćmi i odczytaj wynik jako liczbę dwucyfrową. \nUwaga, w tym przypadku wynik „0” na kostce odczytujemy \nzawsze właśnie jako zero! Zatem rzut 1 na kostce dziesiątek i 4 \nna kostce jedności daje wynik 14, a rzut 4 i 2 oznacza 42. Jeśli \nna obu kościach wypadło 0, wynik wynosi 100.\nKOŚCI ZOSTAŁY RZUCONE\nTomasz Otto (Order #44833549)')

In [110]:
chunks

[Document(metadata={'producer': 'PDFlib+PDI 9.1.1p3 (PHP7/Linux-x86_64)', 'creator': 'PyPDF', 'creationdate': '2024-08-07T07:58:25-04:00', 'source': '/kaggle/input/warhammer-4e-rpg/WFRP_4_ed_PL_1.3.pdf', 'total_pages': 354, 'page': 0, 'page_label': 'I'}, page_content='Tomasz Otto (Order #44833549)'),
 Document(metadata={'producer': 'PDFlib+PDI 9.1.1p3 (PHP7/Linux-x86_64)', 'creator': 'PyPDF', 'creationdate': '2024-08-07T07:58:25-04:00', 'source': '/kaggle/input/warhammer-4e-rpg/WFRP_4_ed_PL_1.3.pdf', 'total_pages': 354, 'page': 1, 'page_label': '1'}, page_content='Tomasz Otto (Order #44833549)'),
 Document(metadata={'producer': 'PDFlib+PDI 9.1.1p3 (PHP7/Linux-x86_64)', 'creator': 'PyPDF', 'creationdate': '2024-08-07T07:58:25-04:00', 'source': '/kaggle/input/warhammer-4e-rpg/WFRP_4_ed_PL_1.3.pdf', 'total_pages': 354, 'page': 2, 'page_label': '2'}, page_content='WARHAMMER FANTASY ROLEPLAY\n2\nSPIS TREŚCI\nWPROWADZENIE\nPowitanie w tej sążnistej księdze, wyjaśnienie podsta-\nwowych reguł,

In [ ]:
('page_content', '27\nniziołki\nNiziołki są wszechobecne w całym Reiklandzie, przeważnie świadczą \nwszelkiej maści usługi we wszystkich miastach. W stolicy Reiklandu, \nAltdorfie, mają nawet swoją dzielnicę, zwaną Haffenstadt. Kłębią się \nw niej setki wielopokoleniowych rodzin niziołków, prowadzących swo-\nje restauracje, karczmy, sklepy z zielem fajkowym i niezliczone uliczne \nkramiki z jedzeniem. Niziołki to powszechny widok także w wielu wio-\nskach Reiklandu, nie dziwota więc, że wielu z nich pracuje w karczmach \nalbo prowadzi własne gospodarstwa. Z natury są bardzo społeczne, \nwolą żyć w zżytych grupach rodzinnych, dzieląc się domami, pokojami \ni łóżkami z mnóstwem przyjaciół i krewnych. Każdy daje coś od siebie \ni dzieli się tym, co ma. Ze względu na styl życia wiele niziołków ma \nproblem z ideą prywatnej własności i przestrzeni.\nNiziołki słyną z zainteresowania genealogią, a wiele klanów może \nwymienić swoich przodków na wiele setek lat wstecz, aż do założenia \nKrainy Zgromadzenia (autonomicznej prowincji niziołków w Im -\nperium). Starszy Krainy Zgromadzenia – aktualnie jest nim Hisme \nStoutheart – jest strażnikiem Haffenlyver, starożytnego, haftowane-\ngo zwoju, szczegółowo opisującego główne linie rodowe pierwot -')

In [107]:
import re

pattern = r"\n.*?\n"
text = "Some text\nRandom content\nAnother section\nMore content\nFinal part."

matches = re.findall(pattern, text)
print(matches)  # Check if it correctly identifies separators


['\nRandom content\n', '\nMore content\n']


## Optimal parameters search

In [ ]:
# TODO
# check optimal chunk size
# check optimal chunk overlap
# check best separators

In [15]:
len(chunks)

801

In [ ]:
('page_content', '•\nA zatem, czego tu szukasz? Przygody?\nByć może. Złota? Zapewne. Sprawiedliwości?\nHa, to dość względne pojęcie! Cóż to? Świętoszkowaty błysk w twym oku? A, chcesz robić \nto, co jest właściwe... Dopóki jest to dobrze płatne, dostarcza Ci rozrywki i pasuje do Twoich \npoglądów. Niech będzie, to wystarczy. Nadasz się. Wejdź, opowiem Ci o tej robocie.\nGdy wymagany jest rezultat losowy, gra WFRP korzysta \nz dziesięciościennej kostki. Dziesięciościenne kostki zazwyczaj \nmają ścianki oznakowane od 0 do 9, gdzie rzut 0 liczy się jako \n10. W zasadach takie kości określane są jako k10, a ich liczba, \nktórą trzeba rzucić, zawsze jest podana w następujący sposób: \n1k10 za jedną kostkę, 2k10 za dwie kostki, 3k10 za trzy kostki \ni tak dalej.\nJeśli należy rzucić kilkoma kośćmi, wyniki są zawsze \nsumowane. Zatem jeśli zasady proszą o rzucenie 2k10, rzucasz \ndwoma dziesięciościennymi kośćmi i dodajesz wyniki ich obu, \nna przykład rzut 0 i 3 oznacza wynik 13 (10+3=13).\nCzasami rzut kostką zostanie zmodyfikowany przez dodanie \nlub odjęcie liczby. Zatem rzut 1k10+4 oznacza rzut jedną \ndziesięciościenną kostką i dodanie do wyniku 4, natomiast rzut \n2k10-3 wskazuje, że należy rzucić dwoma dziesięciościennymi \nkośćmi i odjąć od sumarycznego wyniku 3.\nPonadto zasady wykorzystują rzut dwoma dziesięciościennymi \nkośćmi do uzyskania liczby od 1 do 100 (oznakowane jako \n1k100). Aby to zrobić, jedna dziesięciościenna kostka zostaje \nuznana za kość „dziesiątek”, a druga za kość „jedności”. Teraz \nrzuć dwoma kośćmi i odczytaj wynik jako liczbę dwucyfrową. \nUwaga, w tym przypadku wynik „0” na kostce odczytujemy \nzawsze właśnie jako zero! Zatem rzut 1 na kostce dziesiątek i 4 \nna kostce jedności daje wynik 14, a rzut 4 i 2 oznacza 42. Jeśli \nna obu kościach wypadło 0, wynik wynosi 100.\nKOŚCI ZOSTAŁY RZUCONE\nTomasz Otto (Order #44833549)')